In [1]:
#Figure 6 graph plotting 
#v1.0 version, this is the first consolidation of analysis done so far
#Requires the environment packages found in 23pubenv

In [2]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from skimage import filters, morphology, measure, segmentation
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [3]:
def im_stack_thresh(im_stack): 
    oats = []

    for i in range(len(im_stack)):
        oats.append(filters.threshold_otsu(im_stack[i]))

    return np.mean(oats)

In [6]:
folderpath = Path.cwd().parent / 'Data Sets' / '2023_05_03_CGD_Accumulation' 
folderpath

PosixPath('/Users/echo/Documents/Lab Stuff/Publication/23 Protofigures/Data Sets/2023_05_03_CGD_Accumulation')

In [7]:
#Get all the folders needed:

img_folder = folderpath / 'nuclei'
core_ROI_folder = folderpath / 'nuclei_core_ROI'
seg_folder = folderpath / 'segmented_nuclei'
seg_CORE_folder = folderpath / 'segmented_nuclei_core_ROI'

#Get a list of all the images in each folder: 
img_paths = sorted(img_folder.glob('*.tif'), key= lambda x: x.stem.split('_'))
core_ROI_paths = sorted(core_ROI_folder.glob('*.tif'), key= lambda x: x.stem.split('_'))
seg_paths = sorted(seg_folder.glob('*.tif'), key= lambda x: x.stem.split('_'))
core_ROI_segmentations = sorted(seg_CORE_folder.glob('*.tif'), key= lambda x: x.stem.split('_'))

In [8]:
#For CGD examples: 
#Image 8: img_paths[-2], 18
#Image 11: img_paths[3], 11
#Image 9: img_paths[-1], 4 (zero index)

In [9]:
trains_colors = ['#5BCFFA', '#FFFFFF', '#F5ABB9']
train_palette = LinearSegmentedColormap.from_list(colors=trains_colors, name='trains')

In [19]:

export_location = Path.cwd() / 'accumulation-stills' / 'cgd'
#All images with same stretching: 800-3500
vmin_set = 800
vmax_set = 3000
border_color_core = '#EE362B'
border_color_cells = '#F5ABB9'
border_width = 10


In [20]:
list_position = -1
position_frame = 8

image_name = img_paths[list_position].stem
im_file = tifffile.imread(core_ROI_paths[list_position])[position_frame]
seg_file = tifffile.imread(core_ROI_segmentations[list_position])[position_frame]


In [21]:
#Generating stills with boundry of accumulation overlay on nuclear imaging:

#Get the final core centroid first:

threshold = im_stack_thresh(im_file)

frame = im_file
cell_bin = (seg_file > 0)*1.0

#Threshold core by otsu thresh for stack, then remove all single cells etc
bin_im = frame > threshold
core_adj = morphology.erosion(bin_im)
core_adj = morphology.remove_small_objects(core_adj, 100)
core_adj = morphology.dilation(core_adj)


#Get the info of the cores detected: 
labeled_cores = measure.label(core_adj)
core_infos = pd.DataFrame(measure.regionprops_table(labeled_cores, properties=['label', 'area','centroid']))

#If there are two, take the bigger one as the center one: 
if len(core_infos) > 1: 
    max_idx = core_infos.area.idxmax()
    max_core = core_infos.loc[max_idx]
    core_center = [int(max_core['centroid-1']),int(max_core['centroid-0'])]
    core_mask = labeled_cores == max_core.label
else: 
    core_center = [int(core_infos['centroid-1'].values),int(core_infos['centroid-0'].values)]
    core_mask = core_adj

boundary_core = measure.find_contours(core_mask)

#Plotting
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(frame, cmap='Greys_r', vmin=vmin_set,vmax=vmax_set)
plt.axis('off')
for contour in boundary_core:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=4, fillstyle='full', color=border_color_core)

    
#boundary_cells = measure.find_contours(cell_bin)
#for contour in boundary_cells:
#    ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color=border_color_cells)
plt.gca().invert_yaxis()

scalebar = AnchoredSizeBar(ax.transData,
                           100/1.2865, '',  'lower right', 
                           pad=1,
                           color='white',
                           frameon=False,
                           size_vertical=10)

ax.add_artist(scalebar)
plt.savefig(export_location / Path('cgd-{}-{}.png'.format(image_name, position_frame)), dpi=300, bbox_inches='tight')
plt.close()


In [22]:
max_core.label

2.0

In [23]:
max_core

label             2.000000
area          22948.000000
centroid-0      150.612733
centroid-1      149.223244
Name: 1, dtype: float64